In [9]:
import os
import shutil

import pandas as pd
import tqdm
import src.data.Dataset as dt

In [10]:
FROM_PATH = os.path.abspath("./dataset/emptyNonEmptyDataset")
TO_PATH = os.path.abspath("./dataset/datasetFiltered")

ORIGINAL_DATASET = os.path.abspath("./data/raw/29618Images_binary.csv")
dataset_file_name = os.path.basename(ORIGINAL_DATASET).split("_")[0] + "_filtered.csv"

FILTERED_PATH = os.path.abspath("./data/interim")
FILTERED_DATASET = os.path.join(FILTERED_PATH, dataset_file_name)

print(f"FROM_PATH:         {FROM_PATH}")
print(f"TO_PATH:           {TO_PATH}")
print(f"ORIGINAL_DATASET:  {ORIGINAL_DATASET}")
print(f"FILTERED_PATH:     {FILTERED_PATH}")
print(f"FILTERED_DATASET:  {FILTERED_DATASET}")

FROM_PATH:         /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDataset
TO_PATH:           /Users/carlos/WORKSPACE/MegaClassifier/dataset/datasetFiltered
ORIGINAL_DATASET:  /Users/carlos/WORKSPACE/MegaClassifier/data/raw/29618Images_binary.csv
FILTERED_PATH:     /Users/carlos/WORKSPACE/MegaClassifier/data/interim
FILTERED_DATASET:  /Users/carlos/WORKSPACE/MegaClassifier/data/interim/29618Images_filtered.csv


In [11]:
dataset = dt.load_from_csv(ORIGINAL_DATASET)
dataset

The file /Users/carlos/WORKSPACE/MegaClassifier/data/raw/29618Images_binary.csv has been successfully opened.


,file_name,label
0,\vacia\NOANIMAL_ZOO_5_6_7_9\5_20210219 (614) (...,0
1,\leporido\rabbit_CaltechCameraTrap_ME\58b2edd9...,1
2,\vacia\NOANIMAL_ZOO_5_6_7_9\28_20210417 (2670)...,0
3,\caballo\HORSE_ZOO_5_6_7_9\15_20210219 (8781) ...,1
4,\garduna\garduna_PNC_2014_manuel\c18991IMAG001...,1
...,...,...
29613,\humanovehiculo\humanovehiculo_PNC_2014_manuel...,1
29614,\vacia\NOANIMAL_ZOO_1_4\6_20201116 (3607).JPG,0
29615,\vacia\vacias_PNC_2014_ISAAC\c9150IM000010.JPG,0
29616,\zorro\zorro_PNM_2012_ME\d10536IM000603.JPG,1


In [12]:
total_images_processed = 0
total_images_copied = 0

filtered_dataset = pd.DataFrame({}, columns=dataset.columns)

for _, row in tqdm.tqdm(dataset.iterrows()):
    file_name = row["file_name"].replace("\\", "/")

    original_file = os.path.join(FROM_PATH, file_name[1:])

    filtered_file = file_name.lower()
    filtered_file = filtered_file.replace(" ", "")
    filtered_file = filtered_file.replace("(", "_")
    filtered_file = filtered_file.replace(")", "_")
    filtered_file = (
        filtered_file.replace("á", "a")
        .replace("é", "e")
        .replace("í", "i")
        .replace("ó", "o")
        .replace("ú", "u")
    )
    filtered_file = filtered_file.replace("ñ", "n")

    new_row = pd.DataFrame({"file_name": [filtered_file[1:]], "label": [row["label"]]})
    filtered_dataset = pd.concat([filtered_dataset, new_row], ignore_index=True)

    filtered_file = os.path.join(TO_PATH, filtered_file[1:])

    os.makedirs(os.path.dirname(filtered_file), exist_ok=True)

    try:
        shutil.copyfile(original_file, filtered_file)
        total_images_copied += 1
    except FileNotFoundError:
        print(f"File not found: {original_file}")

    total_images_processed += 1

print(f"Total images processed: {total_images_processed}")
print(f"Total images copied: {total_images_copied}")

dt.dataset_to_csv(filtered_dataset, FILTERED_DATASET)

29618it [00:22, 1343.69it/s]

Total images processed: 29618
Total images copied: 29618
The dataset has been successfully saved to /Users/carlos/WORKSPACE/MegaClassifier/data/interim/29618Images_filtered.csv
